In [1]:
import os
from functools import reduce
import json                                                                     
import random
from collections import OrderedDict
import pdb
from copy import deepcopy
from operator import itemgetter

import pycrfsuite                                                               
import pandas as pd                                                             
import numpy as np    

In [14]:
target_building_name = 'ap_m'
source_building_name = 'ebu3b'
token_type = 'justseparate'
#token_types = ['alphaandnum', 'justseparate']
label_type = 'label'
#label_types = ['label', 'category']
USE_CLUSTER = False
SAMPLE_NUM = 200

data_available_buildings = []

with open('metadata/{0}_char_sentence_dict_{1}.json'.format(target_building_name, token_type), 'r') as fp:
    char_sentence_dict = json.load(fp)
with open('metadata/{0}_sentence_dict_{1}.json'.format(target_building_name, token_type), 'r') as fp:
    word_sentence_dict = json.load(fp)

sentence_dict = char_sentence_dict
crf_model_file = 'model/crf_params_char_{0}_{1}_{2}_{3}_{4}.crfsuite'\
                 .format(source_building_name, token_type, label_type, str(SAMPLE_NUM), \
                         'clustered' if USE_CLUSTER else 'notclustered')

In [15]:
tagger = pycrfsuite.Tagger()
tagger.open(crf_model_file)

In [16]:
def calc_features(sentence, ts_features=None):
    sentenceFeatures = list()
    for i, word in enumerate(sentence):
        features = {
            'word.lower='+word.lower(): 1.0,
            'word.isdigit': float(word.isdigit())
        }
        if i==0:
            features['BOS'] = 1.0
        else:
            features['-1:word.lower=' + sentence[i-1].lower()] = 1.0
            
        if i in [0,1]:
            features['SECOND'] = 1.0
        else:
            features['-2:word.lower=' + sentence[i-2].lower()] = 1.0
            
        if i<len(sentence)-1:
            features['+1:word.lower='+sentence[i+1].lower()] = 1.0
        else:
            features['EOS'] = 1.0
        if ts_features:
            for j, feat in enumerate(ts_features):
                features['ts_feat_'+str(j)] = feat
        sentenceFeatures.append(features)
    return sentenceFeatures

In [17]:
#%%time
predicted_dict = dict()
score_dict = dict()
for srcid, sentence in sentence_dict.items():
    predicted = tagger.tag(calc_features(sentence))
    predicted_dict[srcid] = predicted
    score_dict[srcid] = tagger.probability(predicted)

In [18]:
srcid = '514_0_3006333'
sentence = sentence_dict[srcid]
pred_labels = predicted_dict[srcid]
print(''.join(sentence))
for c, predicted in zip(sentence, pred_labels):
    print('{0}\t{1}'.format(c, predicted))

nae14n2trunk2ahu6hx2mtws
ap&m.hx-2.hx2-mtws
hx2mediumtempwatersupply

n	B_network_adapter
a	I_network_adapter
e	L_network_adapter
1	B_identifier
4	L_identifier
n	U_network_adapter
2	U_identifier
t	U_identifier
r	B_run
u	I_run
n	L_run
k	U_network_adapter
2	U_identifier
a	U_identifier
h	B_network_adapter
u	I_effective
6	L_effective
h	B_heating
x	B_identifier
2	L_identifier
m	B_medium
t	I_medium
w	I_medium
s	L_medium

	O
a	B_damper
p	I_damper
&	I_damper
m	L_damper
.	O
h	B_high
x	L_high
-	O
2	U_identifier
.	O
h	B_heat_exchanger
x	L_heat_exchanger
2	U_identifier
-	O
m	B_medium
t	I_medium
w	I_medium
s	L_medium

	O
h	B_heat_exchanger
x	L_heat_exchanger
2	U_identifier
m	B_mixed_air
e	L_mixed_air
d	B_time
i	I_time
u	I_humidity
m	I_humidity
t	B_temperature
e	I_temperature
m	I_temperature
p	I_temperature
w	I_temperature
a	I_temperature
t	I_temperature
e	I_temperature
r	I_temperature
s	B_supply
u	I_supply
p	I_supply
p	I_supply
l	I_supply
y	L_supply

	O
